In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

In [2]:
import os
os.environ["HF_DATASETS_CACHE"] = "/shared/3/projects/bangzhao/.hf_cache"
from datasets import load_dataset

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torch.optim import AdamW
import torch.nn as nn

from transformers import *
from tokenizers import *

from bertPhoneme import BertEmbeddingsV2, BertModelV2, BertForMaskedLMV2, BertConfigV2, MaskedLMWithProsodyOutput

2025-04-21 14:52:07.422678: I tensorflow/core/platform/cpu_feature_guard.cc:181] Beginning TensorFlow 2.15, this package will be updated to install stock TensorFlow 2.15 alongside Intel's TensorFlow CPU extension plugin, which provides all the optimizations available in the package and more. If a compatible version of stock TensorFlow is present, only the extension will get installed. No changes to code or installation setup is needed as a result of this change.
More information on Intel's optimizations for TensorFlow, delivered as TensorFlow extension plugin can be viewed at https://github.com/intel/intel-extension-for-tensorflow.
2025-04-21 14:52:07.422756: I tensorflow/core/platform/cpu_feature_guard.cc:192] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2025-04-21 14:52:09,792] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/anaconda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/anaconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/opt/anaconda/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@C

In [4]:
import json
import random

In [24]:
file_path = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl"
phonemes = set()

with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 5000:
            break
        phonemes.update(set(json.loads(line)['phoneme']))
    print(len(phonemes))

70


In [25]:
phonemes.update(['UNK', 'SIL'])

In [26]:
phoneme_vocab = {p: i for i, p in enumerate(phonemes)}

In [27]:
print(phoneme_vocab)

{'UW2': 0, 'NG': 1, 'ER0': 2, 'EY2': 3, 'L': 4, 'N': 5, 'ZH': 6, 'spn': 7, 'IH0': 8, 'EH1': 9, 'OW1': 10, 'P': 11, 'AE0': 12, 'Z': 13, 'OW0': 14, 'T': 15, 'HH': 16, 'AA2': 17, 'AW0': 18, 'V': 19, 'UH0': 20, 'B': 21, 'OY1': 22, 'EY1': 23, 'IH1': 24, 'G': 25, 'AA0': 26, 'OW2': 27, 'AE1': 28, 'ER2': 29, 'AO2': 30, 'OY2': 31, 'S': 32, 'R': 33, 'SIL': 34, 'AH1': 35, 'IY1': 36, 'D': 37, 'W': 38, 'CH': 39, 'EH2': 40, 'UH1': 41, 'AH2': 42, 'AE2': 43, 'AA1': 44, 'JH': 45, 'OY0': 46, 'DH': 47, 'UW1': 48, 'SH': 49, 'AY2': 50, 'IY2': 51, 'AY0': 52, 'AH0': 53, 'EY0': 54, 'IY0': 55, 'F': 56, 'M': 57, 'UH2': 58, 'K': 59, 'Y': 60, 'AW2': 61, 'IH2': 62, 'AO1': 63, 'UNK': 64, 'ER1': 65, 'AW1': 66, 'TH': 67, 'UW0': 68, 'AO0': 69, 'EH0': 70, 'AY1': 71}


In [36]:
print(len(phoneme_vocab))

72


In [37]:
with open("/shared/3/projects/bangzhao/prosodic_embeddings/bert_train/phoneme_vocab.json", "w") as f:
    json.dump(phoneme_vocab, f, indent=2)

In [ ]:
with open("/shared/3/projects/bangzhao/prosodic_embeddings/bert_train/phoneme_vocab.json", "r") as f:
    phoneme_vocab = json.load(f)

In [29]:
phoneme_vocab_size = len(phoneme_vocab)
mask_token_id = phoneme_vocab["SIL"]
pad_token_id = 72
pad_cluster_id = 200


class HuggingFacePhonemeDataset(Dataset):
    def __init__(self, hf_dataset, vocab, mask_prob=0.15, max_length=512):
        self.dataset = hf_dataset
        self.vocab = vocab
        self.mask_prob = mask_prob
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        phonemes = sample["phoneme"]
        prosody_ids = sample["prosody_id_200"]

        input_ids = [self.vocab.get(p, self.vocab["UNK"]) for p in phonemes][:self.max_length]
        prosody_ids = prosody_ids[:self.max_length]

        labels = input_ids.copy()
        for i in range(len(input_ids)):
            if random.random() < self.mask_prob:
                labels[i] = input_ids[i]
                input_ids[i] = self.vocab["SIL"]
            else:
                labels[i] = -100

        pad_length = self.max_length - len(input_ids)
        input_ids.extend([pad_token_id] * pad_length)
        labels.extend([-100] * pad_length)
        prosody_ids.extend([pad_cluster_id] * pad_length)
        attention_mask = [1] * (self.max_length - pad_length) + [0] * pad_length
        
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
            "prosody_ids": torch.tensor(prosody_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        }

In [30]:
hf_dataset = load_dataset("json", data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl", split="train")
hf_dataset = hf_dataset.select(range(49000))

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [31]:
train_dataset = HuggingFacePhonemeDataset(hf_dataset, phoneme_vocab)

In [32]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [33]:
vocab_size = phoneme_vocab_size
phoneme_vocab_size = len(phoneme_vocab)
max_length = 512


model_config = BertConfigV2(
    vocab_size=vocab_size+1,
    pad_token_id=pad_token_id,
    pad_cluster_id=200,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=2,
    intermediate_size=512,
    max_position_embeddings=max_length,
    prosody_cluster_size=200 + 1
)

model = BertForMaskedLMV2(config=model_config)

# BERT-Base	768	12	12	3072
# BERT-Small 512	4	8	2048
# BERT-Mini	256	4	4	1024
# BERT-Tiny	128	2	2	512

In [34]:
model

BertForMaskedLMV2(
  (bert): BertModelV2(
    (embeddings): BertEmbeddingsV2(
      (word_embeddings): Embedding(73, 128, padding_idx=72)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (prosody_embeddings): Embedding(201, 128, padding_idx=200)
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

mlm_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
# prosody_loss_fn = nn.CrossEntropyLoss()

num_epochs = 2
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    print(f"\n Epoch {epoch + 1} starting...\n")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        prosody_ids = batch["prosody_ids"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids)

        mlm_loss = mlm_loss_fn(outputs.logits.view(-1, model.config.vocab_size), labels.view(-1))
        # prosody_loss = prosody_loss_fn(outputs.prosody_logits.view(-1, model.config.prosody_cluster_size), prosody_ids.view(-1))

        total_batch_loss = mlm_loss # + prosody_loss
        total_batch_loss.backward()
        optimizer.step()

        total_loss += total_batch_loss.item()

        # Report every 100 steps
        if (step + 1) % 100 == 0:
            avg_loss = total_loss / (step + 1)
            print(f"  Step {step + 1:>4} | Avg Loss: {avg_loss:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    print(f"\n Epoch {epoch + 1} completed. Avg Loss: {avg_epoch_loss:.4f}")

torch.save({
    "model_state_dict": model.state_dict(),
    "config": model_config.to_dict()
}, "/shared/3/projects/bangzhao/prosodic_embeddings/bert_train/mlm_phonemes.pt")


 Epoch 1 starting...

  Step  100 | Avg Loss: 3.2641
  Step  200 | Avg Loss: 2.9354
  Step  300 | Avg Loss: 2.7749
  Step  400 | Avg Loss: 2.6747
  Step  500 | Avg Loss: 2.6055
  Step  600 | Avg Loss: 2.5543
  Step  700 | Avg Loss: 2.5132
  Step  800 | Avg Loss: 2.4792
  Step  900 | Avg Loss: 2.4505
  Step 1000 | Avg Loss: 2.4258
  Step 1100 | Avg Loss: 2.4036
  Step 1200 | Avg Loss: 2.3846
  Step 1300 | Avg Loss: 2.3677
  Step 1400 | Avg Loss: 2.3522
  Step 1500 | Avg Loss: 2.3384

 Epoch 1 completed. Avg Loss: 2.3341

 Epoch 2 starting...

  Step  100 | Avg Loss: 2.1326
  Step  200 | Avg Loss: 2.1286
  Step  300 | Avg Loss: 2.1224
  Step  400 | Avg Loss: 2.1193
  Step  500 | Avg Loss: 2.1162
  Step  600 | Avg Loss: 2.1113
  Step  700 | Avg Loss: 2.1072
  Step  800 | Avg Loss: 2.1024
  Step  900 | Avg Loss: 2.0983
  Step 1000 | Avg Loss: 2.0937
  Step 1100 | Avg Loss: 2.0897
  Step 1200 | Avg Loss: 2.0843
  Step 1300 | Avg Loss: 2.0794
  Step 1400 | Avg Loss: 2.0748
  Step 1500 | Avg

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = torch.load("/shared/3/projects/bangzhao/prosodic_embeddings/bert_train/mlm_phonemes.pt")
config = BertConfigV2(**checkpoint["config"])
model = BertForMaskedLMV2(config=config)
model.load_state_dict(checkpoint["model_state_dict"])

model.to(device)
model.eval()

BertForMaskedLMV2(
  (bert): BertModelV2(
    (embeddings): BertEmbeddingsV2(
      (word_embeddings): Embedding(73, 128, padding_idx=72)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (prosody_embeddings): Embedding(201, 128, padding_idx=200)
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput

In [39]:
hf_dataset_full = load_dataset(
    "json",
    data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl",
    split="train"
)
hf_test_dataset = hf_dataset_full.select(range(49000, 50000))

# === Step 2: Wrap the dataset with HuggingFacePhonemeDataset ===
test_dataset = HuggingFacePhonemeDataset(
    hf_dataset=hf_test_dataset,
    vocab=phoneme_vocab,
    mask_prob=0.15,
    max_length=512
)

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [57]:
sample = test_dataset[2]

input_ids = sample["input_ids"].unsqueeze(0).to(model.device)
labels = sample["labels"].unsqueeze(0).to(model.device)
prosody_ids = sample["prosody_ids"].unsqueeze(0).to(model.device)
attention_mask = sample["attention_mask"].unsqueeze(0).to(model.device)

In [58]:
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]         # [seq_len]
pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]  # [seq_len]

# Evaluation
id2phoneme = {v: k for k, v in phoneme_vocab.items()}
mask_positions = (labels != -100).nonzero(as_tuple=True)[1].tolist()

results = []
correct_phoneme = 0
correct_prosody = 0

for i in mask_positions:
    gt_ph_id = labels[0, i].item()
    pred_ph_id = pred_phonemes[i].item()

    gt_pr_id = prosody_ids[0, i].item()
    pred_pr_id = pred_prosody[i].item()

    phoneme_correct = gt_ph_id == pred_ph_id
    prosody_correct = gt_pr_id == pred_pr_id

    if phoneme_correct:
        correct_phoneme += 1
    if prosody_correct:
        correct_prosody += 1

    results.append({
        "position": i,
        "phoneme_correct": phoneme_correct,
        "phoneme_gt": id2phoneme.get(gt_ph_id, "UNK"),
        "phoneme_pred": id2phoneme.get(pred_ph_id, "UNK"),
        "phoneme_gt_id": gt_ph_id,
        "phoneme_pred_id": pred_ph_id,
        "prosody_correct": prosody_correct,
        "prosody_gt_id": gt_pr_id,
        "prosody_pred_id": pred_pr_id,
    })

# Accuracy
total = len(mask_positions)
phoneme_accuracy = correct_phoneme / total if total > 0 else None
prosody_accuracy = correct_prosody / total if total > 0 else None

print(f"\nPhoneme Accuracy: {phoneme_accuracy:.2%}" if phoneme_accuracy is not None else "No masked phoneme positions.")
print(f"Prosody Accuracy: {prosody_accuracy:.2%}" if prosody_accuracy is not None else "No masked prosody positions.")

for r in results:
    print(f"Pos {r['position']:>3} | Phoneme: Pred={r['phoneme_pred']} GT={r['phoneme_gt']} → {'True' if r['phoneme_correct'] else 'False'}")
    # print(f" | Prosody: Pred={r['prosody_pred_id']} GT={r['prosody_gt_id']} → {'True' if r['prosody_correct'] else 'False'}")



Phoneme Accuracy: 46.91%
Prosody Accuracy: 0.00%
Pos   4 | Phoneme: Pred=DH GT=DH → True
Pos  11 | Phoneme: Pred=K GT=EY1 → False
Pos  25 | Phoneme: Pred=W GT=M → False
Pos  33 | Phoneme: Pred=G GT=G → True
Pos  35 | Phoneme: Pred=D GT=D → True
Pos  36 | Phoneme: Pred=DH GT=T → False
Pos  40 | Phoneme: Pred=W GT=W → True
Pos  43 | Phoneme: Pred=IH1 GT=Y → False
Pos  44 | Phoneme: Pred=AH0 GT=UW1 → False
Pos  45 | Phoneme: Pred=AH0 GT=AH0 → True
Pos  52 | Phoneme: Pred=IY0 GT=IY0 → True
Pos  64 | Phoneme: Pred=ER1 GT=IY1 → False
Pos  75 | Phoneme: Pred=T GT=T → True
Pos  85 | Phoneme: Pred=L GT=N → False
Pos  93 | Phoneme: Pred=AH0 GT=EY1 → False
Pos 101 | Phoneme: Pred=N GT=N → True
Pos 102 | Phoneme: Pred=N GT=Z → False
Pos 107 | Phoneme: Pred=AE1 GT=AH0 → False
Pos 113 | Phoneme: Pred=L GT=L → True
Pos 125 | Phoneme: Pred=F GT=D → False
Pos 127 | Phoneme: Pred=R GT=L → False
Pos 132 | Phoneme: Pred=D GT=D → True
Pos 136 | Phoneme: Pred=T GT=V → False
Pos 142 | Phoneme: Pred=L GT=L →

In [59]:
id2phoneme = {v: k for k, v in phoneme_vocab.items()}

phomenes = []
for input_id in sample['input_ids']:
    phomenes.append(id2phoneme[int(input_id)])

In [61]:
results = [
    {"position": 4, "phoneme_pred": "DH", "phoneme_gt": "DH"},
    {"position": 11, "phoneme_pred": "K", "phoneme_gt": "EY1"},
    {"position": 25, "phoneme_pred": "W", "phoneme_gt": "M"},
    {"position": 33, "phoneme_pred": "G", "phoneme_gt": "G"},
    {"position": 35, "phoneme_pred": "D", "phoneme_gt": "D"},
    {"position": 36, "phoneme_pred": "DH", "phoneme_gt": "T"},
    {"position": 40, "phoneme_pred": "W", "phoneme_gt": "W"},
    {"position": 43, "phoneme_pred": "IH1", "phoneme_gt": "Y"},
    {"position": 44, "phoneme_pred": "AH0", "phoneme_gt": "UW1"},
    {"position": 45, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 52, "phoneme_pred": "IY0", "phoneme_gt": "IY0"},
    {"position": 64, "phoneme_pred": "ER1", "phoneme_gt": "IY1"},
    {"position": 75, "phoneme_pred": "T", "phoneme_gt": "T"},
    {"position": 85, "phoneme_pred": "L", "phoneme_gt": "N"},
    {"position": 93, "phoneme_pred": "AH0", "phoneme_gt": "EY1"},
    {"position": 101, "phoneme_pred": "N", "phoneme_gt": "N"},
    {"position": 102, "phoneme_pred": "N", "phoneme_gt": "Z"},
    {"position": 107, "phoneme_pred": "AE1", "phoneme_gt": "AH0"},
    {"position": 113, "phoneme_pred": "L", "phoneme_gt": "L"},
    {"position": 125, "phoneme_pred": "F", "phoneme_gt": "D"},
    {"position": 127, "phoneme_pred": "R", "phoneme_gt": "L"},
    {"position": 132, "phoneme_pred": "D", "phoneme_gt": "D"},
    {"position": 136, "phoneme_pred": "T", "phoneme_gt": "V"},
    {"position": 142, "phoneme_pred": "L", "phoneme_gt": "L"},
    {"position": 146, "phoneme_pred": "S", "phoneme_gt": "M"},
    {"position": 149, "phoneme_pred": "AH0", "phoneme_gt": "IH0"},
    {"position": 150, "phoneme_pred": "R", "phoneme_gt": "NG"},
    {"position": 152, "phoneme_pred": "N", "phoneme_gt": "N"},
    {"position": 159, "phoneme_pred": "AY1", "phoneme_gt": "AH0"},
    {"position": 160, "phoneme_pred": "DH", "phoneme_gt": "M"},
    {"position": 167, "phoneme_pred": "K", "phoneme_gt": "K"},
    {"position": 172, "phoneme_pred": "NG", "phoneme_gt": "NG"},
    {"position": 183, "phoneme_pred": "N", "phoneme_gt": "N"},
    {"position": 186, "phoneme_pred": "L", "phoneme_gt": "V"},
    {"position": 187, "phoneme_pred": "AH0", "phoneme_gt": "ER0"},
    {"position": 202, "phoneme_pred": "N", "phoneme_gt": "N"},
    {"position": 203, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 213, "phoneme_pred": "T", "phoneme_gt": "K"},
    {"position": 215, "phoneme_pred": "P", "phoneme_gt": "T"},
    {"position": 221, "phoneme_pred": "DH", "phoneme_gt": "P"},
    {"position": 222, "phoneme_pred": "AA1", "phoneme_gt": "R"},
    {"position": 223, "phoneme_pred": "AH0", "phoneme_gt": "EH2"},
    {"position": 233, "phoneme_pred": "ER0", "phoneme_gt": "ER0"},
    {"position": 236, "phoneme_pred": "B", "phoneme_gt": "B"},
    {"position": 241, "phoneme_pred": "K", "phoneme_gt": "NG"},
    {"position": 249, "phoneme_pred": "AH0", "phoneme_gt": "EH1"},
    {"position": 259, "phoneme_pred": "HH", "phoneme_gt": "HH"},
    {"position": 263, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 271, "phoneme_pred": "V", "phoneme_gt": "V"},
    {"position": 280, "phoneme_pred": "EH1", "phoneme_gt": "EH1"},
    {"position": 284, "phoneme_pred": "AA1", "phoneme_gt": "EH2"},
    {"position": 286, "phoneme_pred": "T", "phoneme_gt": "ER0"},
    {"position": 289, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 291, "phoneme_pred": "T", "phoneme_gt": "T"},
    {"position": 304, "phoneme_pred": "AH0", "phoneme_gt": "AH1"},
    {"position": 313, "phoneme_pred": "SH", "phoneme_gt": "Z"},
    {"position": 318, "phoneme_pred": "G", "phoneme_gt": "G"},
    {"position": 320, "phoneme_pred": "N", "phoneme_gt": "D"},
    {"position": 329, "phoneme_pred": "ER0", "phoneme_gt": "ER0"},
    {"position": 332, "phoneme_pred": "B", "phoneme_gt": "B"},
    {"position": 336, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 338, "phoneme_pred": "AH0", "phoneme_gt": "IH1"},
    {"position": 339, "phoneme_pred": "N", "phoneme_gt": "NG"},
    {"position": 349, "phoneme_pred": "V", "phoneme_gt": "V"},
    {"position": 353, "phoneme_pred": "IH0", "phoneme_gt": "IH1"},
    {"position": 370, "phoneme_pred": "N", "phoneme_gt": "T"},
    {"position": 388, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 397, "phoneme_pred": "AE1", "phoneme_gt": "AH0"},
    {"position": 409, "phoneme_pred": "N", "phoneme_gt": "NG"},
    {"position": 417, "phoneme_pred": "D", "phoneme_gt": "AO1"},
    {"position": 431, "phoneme_pred": "EY1", "phoneme_gt": "EY1"},
    {"position": 433, "phoneme_pred": "AH0", "phoneme_gt": "AH0"},
    {"position": 435, "phoneme_pred": "D", "phoneme_gt": "DH"},
    {"position": 436, "phoneme_pred": "AE1", "phoneme_gt": "AE1"},
    {"position": 447, "phoneme_pred": "T", "phoneme_gt": "T"},
    {"position": 448, "phoneme_pred": "AH0", "phoneme_gt": "IH0"},
    {"position": 476, "phoneme_pred": "AE1", "phoneme_gt": "AE1"},
    {"position": 483, "phoneme_pred": "Z", "phoneme_gt": "P"},
    {"position": 486, "phoneme_pred": "IH0", "phoneme_gt": "IH0"},
    {"position": 493, "phoneme_pred": "DH", "phoneme_gt": "DH"},
    {"position": 508, "phoneme_pred": "V", "phoneme_gt": "S"},
]

In [62]:
# Create predicted and ground-truth phoneme sequences
predicted_sentence = phomenes.copy()
gt_sentence = phomenes.copy()

for r in results:
    i = r["position"]
    predicted_sentence[i] = r["phoneme_pred"]
    gt_sentence[i] = r["phoneme_gt"]

# Join for readability
print("Predicted Phoneme Sequence:")
print(" ".join(predicted_sentence))

print("\n Ground Truth Phoneme Sequence:")
print(" ".join(gt_sentence))


Predicted Phoneme Sequence:
HH EH0 L OW1 DH IH1 S IH0 Z G R K AH0 M F ER0 M F AA1 DH ER0 Z L AH1 V W IH1 N AH0 S T R IY0 G UH1 D DH AH0 B IY0 W IH0 TH IH1 AH0 AH0 G EH1 N AE1 T DH IY0 EH1 N D AH0 V AH0 N AH1 DH ER0 W ER1 K W IY1 L UH1 K T Y EH1 S T ER0 D IY0 AO1 N spn K R AA1 L IH0 K AH0 L Z spn D AH0 V IH0 D Z P L AE1 N N F AO1 R DH AE1 T EH1 M P AH0 L AE1 N D D EY1 V IH0 D K AO1 L F AO1 R DH AH0 L IY1 D ER0 Z AH1 T IH1 Z R IY0 AH0 L T UW1 AH0 S IY1 T AH0 R IH0 N JH ER0 UW1 S AH0 L AY1 DH AE1 N D AY1 L AY1 K T AH0 TH IH1 NG K AH0 V IH1 T EH1 Z AH0 HH AE1 N D OW1 L AH0 M IY1 T IH0 NG D EY1 V IH0 HH AE1 D B IH1 N AH0 L AW1 D T UW1 M EY1 K IH0 T S P EH1 N S IH0 V DH AA1 AH0 P ER0 EY1 SH AH0 N Z F R ER0 DH AH0 B IH1 L D IH0 K AH1 V G AA1 D Z T AH0 M P AH0 L B AH1 T HH IY1 HH AE1 D T AH0 HH AE1 N D IH0 T OW1 V ER0 N AW1 T UW1 DH AH0 N EH1 K S JH AA1 N T EY1 SH AH0 N T AH0 S AA1 L AH0 M AH0 N HH IH0 Z S AH0 N HH UW1 W AH0 Z CH OW1 SH AH0 N B AY1 G AA1 N T UW1 R EY1 N AE1 F T ER0 IH0 M B AH1

HH EH0 L OW1          → HELLO  
**DH** IH1 S              → THIS  
IH0 Z                 → IS  
G R **EY1**               → GRAY  
AH0 M                 → UM  
F ER0 M               → FROM  
F AA1 DH ER0 Z        → FATHER'S  
L AH1 V               → LOVE  
**M** IH1 N               → IN  
AH0 S T R IY0 NG      → A STRING  
UH1 D                 → OF GOOD  
**T** AH0 B IY0           → TO BE  
W IH0 TH              → WITH  
**Y** **UW1**                 → YOU  
**AH0** G EH1 N           → AGAIN
AE1 T DH IY0 EH1 N D   → AT THE END  
AH0 V AH0 N AH1        → OF ANOTHER  
DH ER0                 → THEIR  
W **IY1** K                → WEEK  
W IY1 L                → WHILE  
UH1 K T Y EH1 S T ER0 D IY0     → LOOKED YESTERDAY  
AO1 N                      → ON
spn                   → (skip - non-speech noise)
K R AA1 **N** IH0 K AH0 L Z → CHRONICLES
spn D **EY1** V IH0 D         → [skip] DAVID  
Z P L AE1 N Z             → HIS PLANS  
F AO1 R                   → FOR  
DH AH0 T EH1 M P AH0 L    → THE TEMPLE  
AE1 N D D EY1 V IH0 D     → AND DAVID  
K AO1 L D AO1 L           → CALLED ALL  
DH AH0 L IY1 D ER0 Z      → THE LEADERS  
AH1 V IH1 Z              → OF HIS  
R IY0 AH0 L               → ROYAL  
T UW1 AH0 M IY1 T         → TO MEET  
IH0 NG IH0 N              → IN IN  
JH ER0 UW1 S              → JERUSALEM


Hello, this is Gray, from Father's Love. In a string of good,
to be with you again. At the end of another week, while looked yesterday...
David's ordination, chronicles... David planned for the temple,
and called all the leaders of his royal group to meet in Jerusalem.
He had been allowed to make extensive preparations for the building of God's temple.
But he had to hand it over, not to the next generation...

HH EH0 L OW1                        → HELLO  
**DH** IH1 S IH0 Z                     → THIS IS  
G R **K** AH0 M                        → GR(A)COM? [Possibly "GRAHAM" with noisy transcription]  
F ER0 M                            → FROM  
F AA1 DH ER0 Z                     → FATHER’S  
L AH1 V                            → LOVE  
**W** IH1 N                      → WIN
AH0 S T R IY0 G                 → A STRING  
UH1 D                              → OF GOOD  
**DH** AH0 B IY0                       → THE BE  
W IH0 TH                           → WITH  
**IH1** **AH0**                        → IA  (some distortions here)
**AH0** G EH1 N                       → AGAIN
AE1 T DH IY0 EH1 N D               → AT THE END  
AH0 V AH0 N AH1                   → OF ANOTHER  
DH ER0                            → THEIR  
W **ER1** K                            → WORK  
W IY1 L                            → WHILE  
UH1 K T Y EH1 S T ER0 D IY0        → LOOKED YESTERDAY  
AO1 N                              → ON  
K R AA1 **L** IH0 K AH0 L Z           → CHRONICLES
spn D **AH0** V IH0 D                  → [spn] DAVID  
Z P L AE1 N N                      → HIS PLANNING  
F AO1 R DH AE1 T EH1 M P AH0 L     → FOR THAT TEMPLE  
AE1 N D D EY1 V IH0 D              → AND DAVID  
K AO1 L F AO1 R                    → CALLED FOR  
DH AH0 L IY1 D ER0 Z               → THE LEADERS  
AH1 T IH1 Z R IY0 AH0 L            → AT HIS ROYAL  
T UW1 AH0 S IY1 T AH0              → TO A SEAT TO  
R IH0 N JH ER0 UW1 S AH0 L AY1     → IN JERUSALEM


In [66]:
from tqdm import tqdm  # Optional: for progress bar

total_phoneme_correct = 0
total_prosody_correct = 0
total_masked_positions = 0

model.eval()

for idx in tqdm(range(len(test_dataset)), desc="Evaluating samples"):
    sample = test_dataset[idx]

    input_ids = sample["input_ids"].unsqueeze(0).to(model.device)
    labels = sample["labels"].unsqueeze(0).to(model.device)
    prosody_ids = sample["prosody_ids"].unsqueeze(0).to(model.device)
    attention_mask = sample["attention_mask"].unsqueeze(0).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

    pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]
    pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]
    mask_positions = (labels != -100).nonzero(as_tuple=True)[1].tolist()

    for i in mask_positions:
        gt_ph_id = labels[0, i].item()
        pred_ph_id = pred_phonemes[i].item()
        gt_pr_id = prosody_ids[0, i].item()
        pred_pr_id = pred_prosody[i].item()

        total_masked_positions += 1
        if gt_ph_id == pred_ph_id:
            total_phoneme_correct += 1
        if gt_pr_id == pred_pr_id:
            total_prosody_correct += 1

# === Final accuracy calculation ===
if total_masked_positions > 0:
    avg_phoneme_acc = total_phoneme_correct / total_masked_positions
    avg_prosody_acc = total_prosody_correct / total_masked_positions
else:
    avg_phoneme_acc = avg_prosody_acc = None

print(f"\n✅ Evaluated {total_masked_positions} masked positions over {len(test_dataset)} samples")
print(f"Phoneme Accuracy: {avg_phoneme_acc:.2%}" if avg_phoneme_acc is not None else "No masked phoneme positions.")
# print(f"Prosody Accuracy: {avg_prosody_acc:.2%}" if avg_prosody_acc is not None else "No masked prosody positions.")

Evaluating samples: 100%|████████████████████| 1000/1000 [00:22<00:00, 45.14it/s]


✅ Evaluated 74428 masked positions over 1000 samples
Phoneme Accuracy: 43.62%
